In [1]:
'''
To compute CD signatures for LINCS L1000 level3 data using 
all profiles in the same batch as controls
and add results to mongodb.
''' 
import os, sys, json
import h5py
from pymongo import MongoClient
import geode

sys.path.append('/Users/zichen/Documents/bitbucket/maayanlab_utils')
from parallel import *
from fileIO import file2list


sys.path.append('/Users/zichen/Documents/GitHub/l1ktools/python')
import cmap.io.gct as gct


In [2]:
import numpy as np
import pandas as pd

# 0. Load LINCS L1000 level 3 data from gctx file and made function to subset this matrix by probe_ids and sample_ids

In [3]:
GCTX_FILE = '/Users/zichen/Documents/Zichen_Projects/L1000_DR/data/q2norm_n1328098x22268.gctx'
PROBES = json.load(open('/Users/zichen/Documents/bitbucket/lincs_l1000_limma/rid.json', 'rb'))
PROBES_LM1000 = file2list('/Users/zichen/Documents/bitbucket/lincs_l1000_limma/rid_lm1000.txt', 0)

print len(PROBES), len(PROBES_LM1000)
print PROBES[:5]
print PROBES_LM1000[:5]

22268 978
[u'200814_at', u'222103_at', u'201453_x_at', u'204131_s_at', u'200059_s_at']
['200814_at', '222103_at', '201453_x_at', '204131_s_at', '200059_s_at']


In [4]:
gctx = h5py.File(GCTX_FILE, 'r')
mat = gctx['/0/DATA/0/matrix']
print mat.shape

gene_ids = list(gctx['/0/META/ROW/id'])
print len(gene_ids), gene_ids[:5]

(1328098, 22268)
22268 ['200814_at', '222103_at', '201453_x_at', '204131_s_at', '200059_s_at']


In [5]:
gctx['/0/META/COL/id'][:5]

array(['CPC005_A375_6H_X1_B3_DUO52HI53LO:K06',
       'CPC005_A375_6H_X2_B3_DUO52HI53LO:K06',
       'CPC005_A375_6H_X3_B3_DUO52HI53LO:K06',
       'CPC005_A375_6H_X1_B3_DUO52HI53LO:C19',
       'CPC005_A375_6H_X2_B3_DUO52HI53LO:C19'], 
      dtype='|S46')

In [6]:
print np.in1d(PROBES, PROBES_LM1000).sum()
print np.in1d(PROBES, PROBES_LM1000)[:978].sum()

978
978


In [7]:
def slice_matrix(gctx, cids, rids):
    '''Slice the mat by cids and rids and ensure the mat 
    is ordered by cids and rids.'''    
    all_cids = gctx['/0/META/COL/id']
    c_mask = np.in1d(all_cids, cids)
    cids_subset = all_cids[c_mask].tolist()
    c_indices = np.array([cids_subset.index(id_) 
                          for id_ in cids])

    mat = gctx['/0/DATA/0/matrix']
    submat = mat[c_mask, :][c_indices, :]
    
    all_rids = gctx['/0/META/ROW/id']
    r_mask = np.in1d(all_rids, rids)
    rids_subset = all_rids[r_mask].tolist()
    r_indices = np.array([rids_subset.index(id_) 
                          for id_ in rids])
    submat = submat[:, r_mask][:, r_indices]

    return submat

# 1. Connect to MongoDB to retrieve all `sig_id`s that are `trt_cp`

In [8]:
client = MongoClient("mongodb://146.203.54.131:27017")
DB = client['LINCS_L1000_limma']
COLL_SIG = DB['siginfo'] # to get metadata

In [9]:
# Collect pert_ids and batches
cur = COLL_SIG.find({'pert_type':'trt_cp'}, 
                    {'_id':False, 'sig_id':True, 'distil_id':True})
print cur.count()


206339


In [10]:
# Load the metadata for the currently visualized L1000 signatures in L1000FWD
sig_meta_df = pd.read_csv('../data/metadata-full.tsv', sep='\t').set_index('sig_id')
print sig_meta_df.shape
sig_meta_df.head()

(89419, 6)


,cell,dose,pert_id,perturbation,pvalue,time
sig_id,,,,,,
CPC015_MCF7_6H:BRD-A00546892:10.0,MCF7,10.0,BRD-A00546892,-666,0.0143,6
CPC004_VCAP_6H:BRD-A00546892:10.0,VCAP,10.0,BRD-A00546892,BIPERIDEN,0.2056,6
CPC015_ASC_24H:BRD-A00546892:10.0,ASC,10.0,BRD-A00546892,-666,0.2475,24
CPC004_VCAP_24H:BRD-A00546892:10.0,VCAP,10.0,BRD-A00546892,BIPERIDEN,0.3039,24
CPC015_PHH_24H:BRD-A00546892:10.0,PHH,10.0,BRD-A00546892,-666,0.3584,24


In [11]:
print sig_meta_df['pert_id'].nunique()
pert_id_include = set(sig_meta_df['pert_id'].unique())


5312


In [12]:
# Retrieve sig_meta from the MongoDB
cur = COLL_SIG.find({'pert_type':'trt_cp'},
                    {'_id':False, 
                     'sig_id':True, 
                     'distil_id':True,
                    })
sig_meta_df_full = pd.DataFrame([doc for doc in cur]).set_index('sig_id')
print sig_meta_df_full.shape

cur = COLL_SIG.find({'$and': [ 
                        {'pert_type':'trt_cp'},
                        {'distil_nsample': {'$gt': 1}}
                    ]}, 
                    {'_id':False, 
                     'sig_id':True, 
                     'distil_id':True,
                     'pert_id': True,
                     'pert_dose':True,
                     'pert_time':True,
                     'cell_id':True,
                     'pert_desc':True,
                    })
print cur.count()

sig_meta_df = pd.DataFrame([doc for doc in cur if doc['pert_id'] in pert_id_include]).set_index('sig_id')
print sig_meta_df.shape
sig_meta_df.head()

(206339, 1)
202666
(128017, 6)


,cell_id,distil_id,pert_desc,pert_dose,pert_id,pert_time
sig_id,,,,,,
CVD001_HUH7_24H:BRD-K07762753-001-03-6:50,HUH7,"[CVD001_HUH7_24H_X1_F1B3_DUO52HI53LO:M03, CVD0...",-666,50.0,BRD-K07762753,24.0
CPC004_PC3_6H:BRD-K34820100-001-02-1:10,PC3,"[CPC004_PC3_6H_X1_B3_DUO52HI53LO:N23, CPC004_P...",TEBUTHIURON,10.0,BRD-K34820100,6.0
CPC004_PC3_6H:BRD-A22844106-001-16-1:10,PC3,"[CPC004_PC3_6H_X1_B3_DUO52HI53LO:I08, CPC004_P...",TENOXICAM,10.0,BRD-A22844106,6.0
CPC004_PC3_6H:BRD-A55393291-001-05-7:10,PC3,"[CPC004_PC3_6H_X1_B3_DUO52HI53LO:D03, CPC004_P...",TESTOSTERONE,10.0,BRD-A55393291,6.0
CPC004_PC3_6H:BRD-A93255169-001-13-5:10,PC3,"[CPC004_PC3_6H_X1_B3_DUO52HI53LO:F12, CPC004_P...",THALIDOMIDE,10.0,BRD-A93255169,6.0


In [13]:
# Get batch info
sig_meta_df_full['batch'] = sig_meta_df_full.index.map(lambda x:x.split(':')[0])
print sig_meta_df_full['batch'].nunique()

sig_meta_df['batch'] = sig_meta_df.index.map(lambda x:x.split(':')[0])
print sig_meta_df['batch'].nunique()

654
651


# 2. Apply normalization for each batch, then compute signatures and insert in to MongoDB

The normalization method is described in [Iskar et al PLoS comp Bio, 2010]():

```
for batch in batches:
    1. discard control samples
    2. mean center each probe across treatment samples in batch
    3. average replicate treatment samples as the signature
```

In [14]:
# Make MongoDB to store those data
client_local = MongoClient('mongodb://127.0.0.1:27017')
db = client_local['L1000FWD']
coll = db['sigs']
coll.create_index('sig_id', unique=True)

u'sig_id_1'

In [15]:
from joblib import delayed, Parallel

In [16]:
def compute_signatures(sig_id, row, distil_ids_sub, mat, mat_centered, PROBES_LM1000):
    distil_ids_pert = row['distil_id']
    # Make the sample_class
    mask_pert = np.in1d(distil_ids_sub, distil_ids_pert)
    sample_class = mask_pert.astype(int) + 1
    sample_class = list(sample_class)

    # Apply CD on the original mat
    cd1 = geode.chdir(mat.T, sample_class, PROBES_LM1000, 
                      calculate_sig=0, sort=False, gamma=0.5)

    # Apply CD on the mean centered mat
    cd2 = geode.chdir(mat_centered.T, sample_class, PROBES_LM1000, 
                      calculate_sig=0, sort=False, gamma=0.5)
    # Averaging profiles after mean centering
    avg_vals = mat_centered[mask_pert].mean(axis=0)

    doc = row.to_dict()
    doc['sig_id'] = sig_id
    doc['CD_nocenter_LM'] = list(np.array([item[0] for item in cd1], dtype=np.float64))
    doc['CD_center_LM'] = list(np.array([item[0] for item in cd2], dtype=np.float64))
    doc['avg_center_LM'] = list(avg_vals.astype(np.float64))

    return doc

In [ ]:
# Get all inserted document sig_ids
inserted_sig_ids = set(coll.distinct('sig_id'))
print 'Number of sig_ids inserted: %d' % len(inserted_sig_ids)

sig_ids_left = list(set(sig_meta_df.index) - inserted_sig_ids)

# subset the sig_meta_df
sig_meta_df_left = sig_meta_df.ix[sig_ids_left]
all_batches = sig_meta_df_left['batch'].unique()
n_batches = len(all_batches)
print sig_meta_df_left.shape

for c, batch in enumerate(all_batches):
    sig_meta_df_sub = sig_meta_df_left.query('batch == "%s"' % batch)
    
    # all the treatment samples in this batch
    distil_ids_sub = reduce(lambda x, y: x + y, 
                            sig_meta_df_full.query('batch == "%s"' % batch)['distil_id'])
    print c, n_batches
    print '\t', batch, sig_meta_df_sub.shape, len(distil_ids_sub)
    # Slice the matrix
    mat = slice_matrix(gctx, distil_ids_sub, PROBES_LM1000)
    print '\t', mat.shape
    # Mean center the probes
    mat_centered = mat - mat.mean(axis=0)

    docs = Parallel(n_jobs=7, backend='multiprocessing', verbose=10)(\
                              delayed(compute_signatures)(sig_id, row, distil_ids_sub, mat, mat_centered, PROBES_LM1000)\
                              for sig_id, row in sig_meta_df_sub.iterrows())
                              
    result = coll.insert_many(docs)


    


Number of sig_ids inserted: 99420
(28597, 7)
0 328
	CPC006_HT29_24H (358, 7) 1416
	(1416, 978)


[Parallel(n_jobs=7)]: Done   1 out of 358 | elapsed:    5.0s remaining: 29.8min
[Parallel(n_jobs=7)]: Done  36 out of 358 | elapsed:   29.4s remaining:  4.4min
[Parallel(n_jobs=7)]: Done  72 out of 358 | elapsed:   55.3s remaining:  3.7min
[Parallel(n_jobs=7)]: Done 108 out of 358 | elapsed:  1.4min remaining:  3.2min
[Parallel(n_jobs=7)]: Done 144 out of 358 | elapsed:  1.8min remaining:  2.7min
[Parallel(n_jobs=7)]: Done 180 out of 358 | elapsed:  2.2min remaining:  2.2min
[Parallel(n_jobs=7)]: Done 216 out of 358 | elapsed:  2.7min remaining:  1.8min
[Parallel(n_jobs=7)]: Done 252 out of 358 | elapsed:  3.1min remaining:  1.3min
[Parallel(n_jobs=7)]: Done 288 out of 358 | elapsed:  3.6min remaining:   52.9s
[Parallel(n_jobs=7)]: Done 324 out of 358 | elapsed:  4.1min remaining:   25.6s
[Parallel(n_jobs=7)]: Done 358 out of 358 | elapsed:  4.4min finished


1 328
	CPC009_VCAP_6H (363, 7) 1443
	(1443, 978)


[Parallel(n_jobs=7)]: Done   1 out of 363 | elapsed:    4.8s remaining: 29.1min
[Parallel(n_jobs=7)]: Done  32 out of 363 | elapsed:   27.2s remaining:  4.7min
[Parallel(n_jobs=7)]: Done  69 out of 363 | elapsed:   55.5s remaining:  3.9min
[Parallel(n_jobs=7)]: Done 106 out of 363 | elapsed:  1.5min remaining:  3.6min
[Parallel(n_jobs=7)]: Done 143 out of 363 | elapsed:  2.0min remaining:  3.1min
[Parallel(n_jobs=7)]: Done 180 out of 363 | elapsed:  2.5min remaining:  2.5min
[Parallel(n_jobs=7)]: Done 217 out of 363 | elapsed:  3.0min remaining:  2.0min
[Parallel(n_jobs=7)]: Done 254 out of 363 | elapsed:  3.6min remaining:  1.5min
[Parallel(n_jobs=7)]: Done 291 out of 363 | elapsed:  4.1min remaining:  1.0min
[Parallel(n_jobs=7)]: Done 328 out of 363 | elapsed:  4.6min remaining:   29.5s
[Parallel(n_jobs=7)]: Done 363 out of 363 | elapsed:  5.1min finished


2 328
	CPC014_VCAP_6H (344, 7) 1746
	(1746, 978)


...

[Parallel(n_jobs=7)]: Done   1 out of 132 | elapsed:    3.4s remaining:  7.4min
[Parallel(n_jobs=7)]: Done   8 out of 132 | elapsed:    7.1s remaining:  1.8min
[Parallel(n_jobs=7)]: Done  22 out of 132 | elapsed:   14.5s remaining:  1.2min
[Parallel(n_jobs=7)]: Done  36 out of 132 | elapsed:   21.9s remaining:   58.3s
[Parallel(n_jobs=7)]: Done  50 out of 132 | elapsed:   29.3s remaining:   48.0s
[Parallel(n_jobs=7)]: Done  64 out of 132 | elapsed:   36.4s remaining:   38.7s
[Parallel(n_jobs=7)]: Done  78 out of 132 | elapsed:   44.3s remaining:   30.6s
[Parallel(n_jobs=7)]: Done  92 out of 132 | elapsed:   51.3s remaining:   22.3s
[Parallel(n_jobs=7)]: Done 106 out of 132 | elapsed:   58.8s remaining:   14.4s
[Parallel(n_jobs=7)]: Done 120 out of 132 | elapsed:  1.1min remaining:    6.6s
[Parallel(n_jobs=7)]: Done 132 out of 132 | elapsed:  1.2min finished


46 328
	CPC011_VCAP_6H (354, 7) 1749
	(1749, 978)


[Parallel(n_jobs=7)]: Done   1 out of 354 | elapsed:    7.6s remaining: 44.6min
[Parallel(n_jobs=7)]: Done  32 out of 354 | elapsed:   41.9s remaining:  7.0min
[Parallel(n_jobs=7)]: Done  68 out of 354 | elapsed:  1.4min remaining:  5.9min
[Parallel(n_jobs=7)]: Done 104 out of 354 | elapsed:  2.1min remaining:  5.1min
[Parallel(n_jobs=7)]: Done 140 out of 354 | elapsed:  2.8min remaining:  4.3min
[Parallel(n_jobs=7)]: Done 176 out of 354 | elapsed:  3.6min remaining:  3.6min
[Parallel(n_jobs=7)]: Done 212 out of 354 | elapsed:  4.3min remaining:  2.9min
[Parallel(n_jobs=7)]: Done 248 out of 354 | elapsed:  5.0min remaining:  2.2min
[Parallel(n_jobs=7)]: Done 284 out of 354 | elapsed:  5.7min remaining:  1.4min
[Parallel(n_jobs=7)]: Done 320 out of 354 | elapsed:  6.4min remaining:   41.0s
[Parallel(n_jobs=7)]: Done 354 out of 354 | elapsed:  7.1min finished


47 328
	PCLB002_HT29_24H (131, 7) 1097
	(1097, 978)


[Parallel(n_jobs=7)]: Done   1 out of 131 | elapsed:    3.1s remaining:  6.7min
[Parallel(n_jobs=7)]: Done   7 out of 131 | elapsed:    3.9s remaining:  1.1min
[Parallel(n_jobs=7)]: Done  21 out of 131 | elapsed:   11.6s remaining:  1.0min
[Parallel(n_jobs=7)]: Done  35 out of 131 | elapsed:   19.2s remaining:   52.7s
